In [2]:
import numpy as np 
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


import json 


!pip install geocoder
!pip install geopy
from geopy.geocoders import Nominatim 
import geocoder
import geopy

import requests  

from pandas.io.json import json_normalize 

from bs4 import BeautifulSoup

import matplotlib.cm as cm
import matplotlib.colors as colors


!pip install sklearn
from sklearn.cluster import KMeans

!pip install folium
import folium 

print('Libraries imported.')

Libraries imported.


In [3]:
import csv

website_url = requests.get('https://en.wikipedia.org/wiki/Demographics_of_Berlin').text
soup = BeautifulSoup(website_url,'lxml')
table = soup.find('table',{'class':'wikitable sortable'})
#print(soup.prettify())

headers = [header.text for header in table.find_all('th')]

table_rows = table.find_all('tr')        
rows = []
for row in table_rows:
   td = row.find_all('td')
   row = [row.text for row in td]
   rows.append(row)


with open('df1.csv', 'w') as f:
   writer = csv.writer(f)
   writer.writerow(headers)
   writer.writerows(row for row in rows if row)

In [5]:
df1=pd.read_csv('df1.csv',encoding= 'unicode_escape')
df1

,Borough,Population 2010,Area in km²,Largest Non-German ethnic groups\r\n
0,Mitte,"332,100",39.47,"Turks, Arabs, Kurds, Asians, Western Europeans..."
1,Friedrichshain-Kreuzberg,"268,831",20.16,"Turks, Arabs, Africans, Kurds, Chinese\r\n"
2,Pankow,"368,956",103.01,"Poles, Italians, French, Americans, Vietnamese..."
3,Charlottenburg-Wilmersdorf,"320,014",64.72,"Turks, Africans, Russians, Arabs\r\n"
4,Spandau,"225,420",91.91,"Turks, Africans, Russians, Arabs\r\n"
5,Steglitz-Zehlendorf,"293,989",102.50,"Poles, Turks, Croats, Serbs, Koreans\r\n"
6,Tempelhof-Schöneberg,"335,060",53.09,"Turks, Croats, Serbs, Koreans, Africans\r\n"
7,Neukölln,"310,283",44.93,"Arabs, Turks, Kurds, Russians, Africans, Poles..."
8,Treptow-Köpenick,"241,335",168.42,"Russians, Poles, Ukrainians, Vietnamese\r\n"
9,Marzahn-Hellersdorf,"248,264",61.74,"Russians, Vietnamese, Eastern Europeans\r\n"


In [6]:
# Geographical coordinates of neighborhoods

# define a function to get coordinates
def get_latlng(borough):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Berlin, Germany'.format(borough))
        lat_lng_coords = g.latlng
    return lat_lng_coords

coords = [ get_latlng(borough) for borough in df1["Borough"].tolist() ]

df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

# merge the coordinates into the original dataframe
df1['Latitude'] = df_coords['Latitude']
df1['Longitude'] = df_coords['Longitude']


# check the neighborhoods and the coordinates
print(df1.shape)
df1.head()

(13, 6)


,Borough,Population 2010,Area in km²,Largest Non-German ethnic groups\r\n,Latitude,Longitude
0,Mitte,"332,100",39.47,"Turks, Arabs, Kurds, Asians, Western Europeans...",52.52119,13.42414
1,Friedrichshain-Kreuzberg,"268,831",20.16,"Turks, Arabs, Africans, Kurds, Chinese\r\n",52.50061,13.41863
2,Pankow,"368,956",103.01,"Poles, Italians, French, Americans, Vietnamese...",52.56925,13.40248
3,Charlottenburg-Wilmersdorf,"320,014",64.72,"Turks, Africans, Russians, Arabs\r\n",52.49962,13.32316
4,Spandau,"225,420",91.91,"Turks, Africans, Russians, Arabs\r\n",52.53487,13.20216


In [8]:

# save the DataFrame as CSV file
df1.to_csv("berlin_neighborhoods.csv", index=False)

In [9]:
address = 'Berlin, Germany'

geolocator = Nominatim(user_agent = "myapp1")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Berlin is {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Berlin is 52.5170365, 13.3888599.


In [10]:
# create map of Berlin using latitude and longitude values
map_berlin = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough in zip(df1['Latitude'], df1['Longitude'], df1['Borough']):
    label = '{}'.format(borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_berlin)  
    
map_berlin

## Using Foursquare API

In [11]:
CLIENT_ID = 'YP03FATPA3WT3GI4K5D5EC2XOTUE3YKFYULHSR1DBJSIKK5M' # your Foursquare ID
CLIENT_SECRET = 'FQTXNXDDJYMZVYAUHCKY5TVVF1WZPJKLDMI411XXAGYOHV2P' # your Foursquare Secret
ACCESS_TOKEN = '42GH2CXALXEM5R1VEXYBCUVJETZ31E135YWUNA1DSPDPFMTT' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 50
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: YP03FATPA3WT3GI4K5D5EC2XOTUE3YKFYULHSR1DBJSIKK5M
CLIENT_SECRET:FQTXNXDDJYMZVYAUHCKY5TVVF1WZPJKLDMI411XXAGYOHV2P


In [12]:
radius = 1500
LIMIT = 50

venues = []
for lat, long, neighborhood in zip(df1['Latitude'], df1['Longitude'], df1['Borough']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [13]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Borough', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(530, 7)


,Borough,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Mitte,52.52119,13.42414,Kino International,52.520363,13.422849,Indie Movie Theater
1,Mitte,52.52119,13.42414,Ristorante a Mano,52.518772,13.427652,Italian Restaurant
2,Mitte,52.52119,13.42414,CAMP4,52.520032,13.421953,Outdoor Supply Store
3,Mitte,52.52119,13.42414,Singer109 - Hostel Berlin,52.516631,13.425505,Hostel
4,Mitte,52.52119,13.42414,Märchenbrunnen,52.527999,13.426752,Fountain


In [14]:

venues_df.groupby(["Borough"]).count()
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 169 uniques categories.


In [15]:
#### Analyse each neighborhood
# one hot encoding
onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
onehot['Boroughs'] = venues_df['Borough'] 

# move neighborhood column to the first column
fixed_columns = [onehot.columns[-1]] + list(onehot.columns[:-1])
onehot = onehot[fixed_columns]

print(onehot.shape)

grouped = onehot.groupby(["Boroughs"]).mean().reset_index()

print(grouped.shape)
grouped.head()

(530, 170)
(13, 170)


,Boroughs,African Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,Automotive Shop,Baby Store,Bagel Shop,Bakery,Bank,Bar,Bathing Area,Bed & Breakfast,Beer Bar,Beer Garden,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Bookstore,Boutique,Breakfast Spot,Brewery,Burger Joint,Burrito Place,Bus Stop,Café,Candy Store,Caribbean Restaurant,Castle,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comic Shop,Cooking School,Cosmetics Shop,Costume Shop,Deli / Bodega,Department Store,Dessert Shop,Dive Bar,Doner Restaurant,Drugstore,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Food & Drink Shop,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gas Station,General Entertainment,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Historic Site,History Museum,Hostel,Hotel,Ice Cream Shop,Imported Food Shop,Indie Movie Theater,Indie Theater,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Kebab Restaurant,Korean Restaurant,Kurdish Restaurant,Light Rail Station,Liquor Store,Lounge,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Neighborhood,Nightclub,Noodle House,Optical Shop,Organic Grocery,Outdoor Supply Store,Palace,Paper / Office Supplies Store,Park,Pastry Shop,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Piercing Parlor,Pizza Place,Playground,Plaza,Pool,Portuguese Restaurant,Post Office,Pub,Record Shop,Recreation Center,Residential Building (Apartment / Condo),Restaurant,Roof Deck,Scenic Lookout,Seafood Restaurant,Shopping Mall,Soccer Field,Spa,Spanish Restaurant,Sporting Goods Shop,Squash Court,Stationery Store,Steakhouse,Street Food Gathering,Supermarket,Sushi Restaurant,Syrian Restaurant,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Trail,Tram Station,Trattoria/Osteria,Tree,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo Exhibit
0,Charlottenburg-Wilmersdorf,0.00,0.0,0.00,0.02,0.000000,0.00,0.0,0.0,0.0,0.06,0.0,0.00,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.02,0.00,0.00,0.0,0.02,0.00,0.000,0.02,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.02,0.00,0.00,0.00,0.0,0.00,0.0,0.04,0.00,0.02,0.020000,0.0,0.0,0.0,0.02,0.00,0.00,0.00,0.0,0.0,0.00,0.000000,0.00,0.0,0.0,0.00,0.04,0.00,0.0,0.00,0.00,0.00,0.02,0.0,0.0,0.000000,0.00,0.0,0.0,0.00,0.0,0.00,0.120000,0.000000,0.00,0.00,0.0,0.00,0.02,0.04,0.02,0.00,0.0,0.00,0.000000,0.000000,0.00,0.0,0.02,0.00,0.000000,0.00,0.02,0.0,0.0,0.0,0.00,0.04,0.04,0.02,0.0,0.00,0.00,0.0,0.00,0.04,0.00,0.00,0.00,0.0,0.0,0.00,0.02,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.04,0.000000,0.02,0.0,0.00,0.00,0.0,0.0,0.02,0.00,0.0,0.02,0.000000,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.02,0.020000,0.02,0.00,0.0,0.0,0.00,0.02,0.000000,0.0,0.0,0.0,0.00,0.0,0.02,0.02,0.00,0.04,0.02,0.00,0.0
1,Friedrichshain-Kreuzberg,0.02,0.0,0.02,0.00,0.000000,0.00,0.0,0.0,0.0,0.00,0.0,0.04,0.0,0.00,0.02,0.0,0.0,0.00,0.00,0.02,0.02,0.02,0.02,0.00,0.0,0.02,0.00,0.000,0.04,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.04,0.12,0.00,0.02,0.00,0.00,0.0,0.00,0.0,0.00,0.02,0.00,0.000000,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.0,0.02,0.000000,0.00,0.0,0.0,0.00,0.00,0.02,0.0,0.02,0.04,0.00,0.02,0.0,0.0,0.000000,0.00,0.0,0.0,0.00,0.0,0.00,0.000000,0.000000,0.00,0.04,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.02,0.000000,0.000000,0.02,0.0,0.00,0.00,0.000000,0.00,0.04,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.02,0.02,0.0,0.00,0.00,0.00,0.02,0.00,0.0,0.0,0.00,0.00,0.02,0.00,0.0,0.02,0.00,0.02,0.00,0.00,0.000000,0.00,0.0,0.00,0.04,0.0,0.0,0.00,0.00

In [16]:
len(grouped[grouped["Italian Restaurant"] > 0])
berlin_rest = grouped[["Boroughs","Italian Restaurant","Bar"]]
berlin_rest

,Boroughs,Italian Restaurant,Bar
0,Charlottenburg-Wilmersdorf,0.000000,0.00
1,Friedrichshain-Kreuzberg,0.000000,0.04
2,Lichtenberg,0.080000,0.02
3,Marzahn-Hellersdorf,0.000000,0.00
4,Mitte,0.020000,0.00
5,Neukölln,0.000000,0.02
6,Pankow,0.040000,0.02
7,Reinickendorf,0.000000,0.00
8,Spandau,0.040000,0.00
9,Steglitz-Zehlendorf,0.140000,0.00


## Clustering

In [17]:
kclusters = 4

berlin_clustering = berlin_rest.drop(["Boroughs"], 1)
berlin_clustering.head()

# run k-means clustering
kmeans = KMeans(init="k-means++", n_clusters=kclusters, n_init=12).fit(berlin_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([3, 3, 0, 3, 1, 3, 1, 3, 1, 2])

In [18]:
merged = berlin_rest.copy()

# add clustering labels
merged["Category"] = kmeans.labels_
merged.rename(columns={"Boroughs": "Borough"}, inplace=True)
merged.head()

,Borough,Italian Restaurant,Bar,Category
0,Charlottenburg-Wilmersdorf,0.00,0.00,3
1,Friedrichshain-Kreuzberg,0.00,0.04,3
2,Lichtenberg,0.08,0.02,0
3,Marzahn-Hellersdorf,0.00,0.00,3
4,Mitte,0.02,0.00,1


In [19]:
dfmerged = merged.merge(df1)
dfmerged.head()

#Sort
dfmerged.sort_values(["Category"], inplace=True, ascending=False)
dfmerged

,Borough,Italian Restaurant,Bar,Category,Population 2010,Area in km²,Largest Non-German ethnic groups\r\n,Latitude,Longitude
0,Charlottenburg-Wilmersdorf,0.000000,0.00,3,"320,014",64.72,"Turks, Africans, Russians, Arabs\r\n",52.499620,13.323160
1,Friedrichshain-Kreuzberg,0.000000,0.04,3,"268,831",20.16,"Turks, Arabs, Africans, Kurds, Chinese\r\n",52.500610,13.418630
3,Marzahn-Hellersdorf,0.000000,0.00,3,"248,264",61.74,"Russians, Vietnamese, Eastern Europeans\r\n",52.516670,13.583330
5,Neukölln,0.000000,0.02,3,"310,283",44.93,"Arabs, Turks, Kurds, Russians, Africans, Poles...",52.480770,13.435410
7,Reinickendorf,0.000000,0.00,3,"240,454",89.46,"Turks, Poles, Serbs, Croats, Arabs, Italians\r\n",52.575450,13.349700
10,Tempelhof-Schöneberg,0.000000,0.00,3,"335,060",53.09,"Turks, Croats, Serbs, Koreans, Africans\r\n",52.477706,13.358674
12,Treptow-Köpenick,0.000000,0.00,3,"241,335",168.42,"Russians, Poles, Ukrainians, Vietnamese\r\n",52.433330,13.600000
9,Steglitz-Zehlendorf,0.140000,0.00,2,"293,989",102.50,"Poles, Turks, Croats, Serbs, Koreans\r\n",52.434850,13.241830
4,Mitte,0.020000,0.00,1,"332,100",39.47,"Turks, Arabs, Kurds, Asians, Western Europeans...",52.521190,13.424140
6,Pankow,0.040000,0.02,1,"368,956",103.01,"Poles, Italians, French, Americans, Vietnamese...",52.569250,13.402480


In [20]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(dfmerged['Latitude'], dfmerged['Longitude'], dfmerged['Borough'], dfmerged['Category']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [26]:
merged.loc[merged['Category'] == 0]

,Borough,Italian Restaurant,Bar,Category
2,Lichtenberg,0.080000,0.02,0
11,Total Berlin,0.090909,0.00,0


In [27]:
merged.loc[merged['Category'] == 1]

,Borough,Italian Restaurant,Bar,Category
4,Mitte,0.02,0.00,1
6,Pankow,0.04,0.02,1
8,Spandau,0.04,0.00,1


In [28]:
merged.loc[merged['Category'] == 2]

,Borough,Italian Restaurant,Bar,Category
9,Steglitz-Zehlendorf,0.14,0.0,2


In [29]:
merged.loc[merged['Category'] == 3]

,Borough,Italian Restaurant,Bar,Category
0,Charlottenburg-Wilmersdorf,0.0,0.00,3
1,Friedrichshain-Kreuzberg,0.0,0.04,3
3,Marzahn-Hellersdorf,0.0,0.00,3
5,Neukölln,0.0,0.02,3
7,Reinickendorf,0.0,0.00,3
10,Tempelhof-Schöneberg,0.0,0.00,3
12,Treptow-Köpenick,0.0,0.00,3


In [30]:
import csv

website_url = requests.get('https://en.wikipedia.org/wiki/Demographics_of_Berlin').text
soup = BeautifulSoup(website_url,'lxml')
table = soup.find('table',{'class':'wikitable sortable'})
#print(soup.prettify())

#headers = [header.text for header in table.find_all('th')]

table_rows = table.find_all('tr')        
rows = []
for row in table_rows:
   td = row.find_all('td')
   row = [row.text for row in td]
   rows.append(row)


with open('df1.csv', 'w') as f:
   writer = csv.writer(f)
   writer.writerow(headers)
   writer.writerows(row for row in rows if row)



In [31]:
df1=pd.read_csv('df1.csv',encoding= 'unicode_escape')
df1

,Borough,Population 2010,Area in km²,Largest Non-German ethnic groups\r\n
0,Mitte,"332,100",39.47,"Turks, Arabs, Kurds, Asians, Western Europeans..."
1,Friedrichshain-Kreuzberg,"268,831",20.16,"Turks, Arabs, Africans, Kurds, Chinese\r\n"
2,Pankow,"368,956",103.01,"Poles, Italians, French, Americans, Vietnamese..."
3,Charlottenburg-Wilmersdorf,"320,014",64.72,"Turks, Africans, Russians, Arabs\r\n"
4,Spandau,"225,420",91.91,"Turks, Africans, Russians, Arabs\r\n"
5,Steglitz-Zehlendorf,"293,989",102.50,"Poles, Turks, Croats, Serbs, Koreans\r\n"
6,Tempelhof-Schöneberg,"335,060",53.09,"Turks, Croats, Serbs, Koreans, Africans\r\n"
7,Neukölln,"310,283",44.93,"Arabs, Turks, Kurds, Russians, Africans, Poles..."
8,Treptow-Köpenick,"241,335",168.42,"Russians, Poles, Ukrainians, Vietnamese\r\n"
9,Marzahn-Hellersdorf,"248,264",61.74,"Russians, Vietnamese, Eastern Europeans\r\n"


In [32]:
df2 = df1[df1['Largest Non-German ethnic groups\r\n'].str.contains('Italian',regex=False)]
df2.columns

Index(['Borough', 'Population 2010', 'Area in km²',
       'Largest Non-German ethnic groups\r\n'],
      dtype='object')

In [33]:
df2.drop(['Population 2010', 'Area in km²'],axis=1, inplace=True)
df2

C:\Users\OBO\anaconda3\lib\site-packages\pandas\core\frame.py:4167: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,Borough,Largest Non-German ethnic groups\r\n
2,Pankow,"Poles, Italians, French, Americans, Vietnamese..."
11,Reinickendorf,"Turks, Poles, Serbs, Croats, Arabs, Italians\r\n"


In [34]:
dfmerged = dfmerged.merge(df2)
dfmerged

,Borough,Italian Restaurant,Bar,Category,Population 2010,Area in km²,Largest Non-German ethnic groups\r\n,Latitude,Longitude
0,Reinickendorf,0.00,0.00,3,"240,454",89.46,"Turks, Poles, Serbs, Croats, Arabs, Italians\r\n",52.57545,13.34970
1,Pankow,0.04,0.02,1,"368,956",103.01,"Poles, Italians, French, Americans, Vietnamese...",52.56925,13.40248


In [37]:
dfmerged.rename(columns = {'Population 2010': 'Population', 'Largest Non-German ethnic groups\r\n': 'Largest Non-German ethnic groups'}, inplace = True)

In [39]:
dfmerged

,Borough,Italian Restaurant,Bar,Category,Population,Area in km²,Largest Non-German ethnic groups,Latitude,Longitude
0,Reinickendorf,0.00,0.00,3,"240,454",89.46,"Turks, Poles, Serbs, Croats, Arabs, Italians\r\n",52.57545,13.34970
1,Pankow,0.04,0.02,1,"368,956",103.01,"Poles, Italians, French, Americans, Vietnamese...",52.56925,13.40248
